1. Generate a signature:
<br><br>
$e * G = P, z = hash(M)$, k is a random number unique for each signature.
<br><br>
$k * G = u * G + v * P$
<br><br>
$k * G = u * G + v * e * G$
<br><br>
$k = u + v * e$
<br><br>
Let $r = (k * G).x, u = \frac{z}{s}, v = \frac{r}{s}$, then
<br><br>
$k = \frac{z}{s} + \frac{r * e}{s}$
<br><br>
$k = \frac{z + r * e}{s}$
<br><br>
$s = \frac{z + r * e}{k}$
<br><br>
$Signature = (r, s)$
<br><br>
2. Verify a signature:
<br><br>
$u = \frac{z}{s}$
<br><br>
$v = \frac{r}{s}$
<br><br>
Verify that $(u * G + v * P).x == r$
<br><br>
3. Can the signer reveal k?
<br><br>
No. Since $k = u + v * e$, revealing k means revealing the private key e.
<br><br>
4. Can the signer reveal r?
<br><br>
Yes. Since $k * G = (r, y)$, kowing r and y, it's still hard to find k becasue it's an elliptic curve discrete logarithm problem.
<br><br>
5. What if k is not unique for each signature?
<br><br>
Private key e can be revealed.
<br><br>
$k * G = (r, y)$
<br><br>
$s_{1} = \frac{z_{1} + r * e}{k}, s_{2} = \frac{z_{2} + r * e}{k}$
<br><br>
$\frac{s_{1}}{s_{2}} = \frac{z_{1} + r * e}{z_{2} + r * e}$
<br><br>
$s_{1} * (z_{2} + r * e) = s_{2} * (z_{1} + r * e)$
<br><br>
$s_{1} * z_{2} + s_{1} * r * e = s_{2} * z_{1} + s_{2} * r * e$
<br><br>
$s_{1} * r * e - s_{2} * r * e = s_{2} * z_{1} - s_{1} * z_{2}$
<br><br>
$e = \frac{s_{2} * z_{1} - s_{1} * z_{2}}{s_{1} * r - s_{2} * r}$

In [1]:
from hashlib import sha256
import hmac

class FieldElement:
    def __init__(self, num, prime):
        if num >= prime or num < 0:
            raise ValueError("Number {0} is not in the range between 0 and {1} - 1".format(num, prime))
        self.num = num
        self.prime = prime
        
    def __eq__(self, other):
        if not other:
            return False
        elif self.num == other.num and self.prime == other.prime:
            return True
        else:
            return False
        
    def __ne__(self, other):
        if not other:
            return False
        elif self.num != other.num or self.prime != other.prime:
            return True
        else:
            return False
        
    def __repr__(self):
        return "FieldElement_{0}({1})".format(self.prime, self.num)
    
    def __add__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot add two numbers in different fields.")
        num = (self.num + other.num) % self.prime
        return self.__class__(num, self.prime)
    
    def __sub__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot add two numbers in different fields.")
        num = (self.num - other.num) % self.prime
        return self.__class__(num, self.prime)
    
    def __mul__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot add two numbers in different fields.")
        num = (self.num * other.num) % self.prime
        return self.__class__(num, self.prime)
    
    def __rmul__(self, scalor):
        num = (self.num * scalor) % self.prime
        return self.__class__(num, self.prime)
    
    def __truediv__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot add two numbers in different fields.")
        num = (self.num * pow(other.num, other.prime - 2, other.prime)) % self.prime
        return self.__class__(num, self.prime)
    
    def __pow__(self, exponent):
        n = exponent % (self.prime - 1)
        num = pow(self.num, n, self.prime)
        return self.__class__(num, self.prime)
    
class Point:
    def __init__(self, x, y, a, b):
        self.x = x
        self.y = y
        self.a = a
        self.b = b
        if self.x == None and self.y == None:
            return 
        if self.y ** 2 != self.x ** 3 + a * x + b:
            raise ValueError("({0}, {1}) is not on the curve.".format(x, y))
            
    def __repr__(self):
        if self.x is None and self.y is None:
            return "Point(Infinity)"
        elif isinstance(self.x, FieldElement):
            return "Point({0}, {1})_{2}_{3} FieldElement({4})".format(self.x.num, self.y.num, self.a.num, self.b.num, self.x.prime)
        else:
            return "Point({0}, {1})_{2}_{3}".format(self.x, self.y, self.a, self.b)
        
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y and self.a == other.a and self.b == other.b
    
    def __ne__(self, other):
        return self.x != other.x or self.y != other.y or self.a != other.a or self.b != other.b
    
    def __add__(self, other):
        if self.a != other.a or self.b != other.b:
            raise TypeError("Point {0} and {1} are not on the same curve".format(self, other))
            
        if self.x is None:
            return other
        
        if other.x is None:
            return self
        
        if self.x == other.x and self.y != other.y:
            return self.__class__(None, None, self.a, self.b)
        
        if self.x != other.x:
            s = (other.y - self.y)/(other.x - self.x)
            x = s ** 2 - self.x - other.x
            y = s * (self.x - x) - self.y
            return self.__class__(x, y, self.a, self.b)
        
        if self == other:
            if self.y == 0:
                return self.__class__(None, None, self.a, self.b)
            else:
                s = (3 * self.x ** 2 + self.a) / (2 * self.y)
                x = s ** 2 - 2 * self.x
                y = s * (self.x - x) - self.y
                return self.__class__(x, y, self.a, self.b)
    
    def __rmul__(self, scalor):
        s = scalor
        current = self
        res = self.__class__(None, None, self.a, self.b)
        while s:
            if s & 1:
                res += current
            current += current 
            s >>= 1
        return res

class S256Field(FieldElement):
    def __init__(self, num, prime = None):
        super().__init__(num = num, prime = P)
        
class S256Point(Point):
    def __init__(self, x, y, a = None, b = None):
        a, b = S256Field(A), S256Field(B)
        if type(x) == int:
            super().__init__(S256Field(x), S256Field(y), a, b)
        else:
            super().__init__(x, y, a, b)
    
    def __repr__(self):
        if self.x is None and self.y is None:
            return "S256Point(Infinity)"
        else:
            return "S256Point({0}, {1})".format(self.x, self.y)
        
    def __rmul__(self, scalor):
        s = scalor % N
        return super().__rmul__(s)
    
    def verify(self, z, sig):
        r, s = sig.r, sig.s
        s_inv = pow(s, N - 2, N)
        u = z * s_inv % N
        v = r * s_inv % N
        return (u * G + v * self).x.num == r
    
### Define secp256k1 ###
A = 0
B = 7
P = 2**256 - 2**32 - 977
X = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
Y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
N = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
G = S256Point(X, Y)

class Signature:
    def __init__(self, r, s):
        self.r = r
        self.s = s

class PrivateKey:
    def __init__(self, e):
        self.secret = e
        self.public_key = self.secret * G
        
    def sign(self, z):
        k = self.deterministic_k(z)
        r = (k * G).x.num
        k_inv = pow(k, N - 2, N)
        s = (z + r * self.secret) * k_inv % N
        if s > N / 2:
            s = N - s
        return Signature(r, s)
    
    def deterministic_k(self, z):
        k = b'\x00' * 32
        v = b'\x01' * 32
        if z > N:
            z -= N
        z_bytes = z.to_bytes(32, 'big')
        secret_bytes = self.secret.to_bytes(32, 'big')
        k = hmac.new(k, v + b'\x00' + secret_bytes + z_bytes, sha256).digest()
        v = hmac.new(k, v, sha256).digest()
        k = hmac.new(k, v + b'\x01' + secret_bytes + z_bytes, sha256).digest()
        v = hmac.new(k, v, sha256).digest()
        while True:
            v = hmac.new(k, v, sha256).digest()
            candidate = int.from_bytes(v, 'big')
            if candidate >= 1 and candidate < N:
                return candidate
            k = hmac.new(k, v + b'\x00', sha256).digest()
            v = hmac.new(k, v, sha256).digest()
    
def hash_message(M):
    return int.from_bytes(sha256(sha256(M.encode("utf-8")).digest()).digest(), "big")

In [2]:
from random import SystemRandom
_sysrand = SystemRandom()

z = hash_message("Hello, how are you?")
private_key = PrivateKey(_sysrand.getrandbits(256))
sig = private_key.sign(z)
private_key.public_key.verify(z, sig)

True

In [3]:
z2 = hash_message("I'm fine. Thank you.")
private_key.public_key.verify(z2, sig)

False

In [4]:
private_key_2 = PrivateKey(_sysrand.getrandbits(256))
private_key_2.public_key.verify(z, sig)

False

#### Reference: "Programming Bitcoin: Learn How to Program Bitcoin from Scratch", 1st Edition, 2019